# LangChain Chat Application with Claude

This notebook demonstrates a sophisticated LLM chat application using LangChain and Anthropic's Claude model.

## Features:
- Advanced prompt engineering with system and human messages
- Conversational chains with memory
- Multiple use cases: Q&A, creative writing, code generation
- Streaming responses
- Error handling
- Multi-turn dialogues with context awareness

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from typing import List, Dict, Any

# LangChain imports
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

print("✓ All libraries imported successfully")

In [ ]:
# Load environment variables
load_dotenv()

# Verify API keys are loaded
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
langsmith_key = os.getenv("LANGSMITH_API_KEY")

if not anthropic_key:
    raise ValueError("ANTHROPIC_API_KEY not found. Please create a .env file with your API key.")

print("✓ Environment variables loaded successfully")
print(f"✓ Anthropic API Key: {'*' * 20}{anthropic_key[-4:] if anthropic_key else 'Not set'}")
print(f"✓ LangSmith tracking: {'Enabled' if langsmith_key else 'Disabled (optional)'}")

## 2. Initialize Chat Model

In [ ]:
# Initialize Claude model with various configurations
def create_chat_model(
    model_name: str = "claude-3-5-sonnet-20241022",
    temperature: float = 0.7,
    max_tokens: int = 4096
) -> ChatAnthropic:
    """Create and configure a Claude chat model."""
    return ChatAnthropic(
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        anthropic_api_key=anthropic_key
    )

# Create default model instance
chat_model = create_chat_model()
print(f"✓ Chat model initialized: {chat_model.model}")
print(f"  - Temperature: {chat_model.temperature}")
print(f"  - Max tokens: {chat_model.max_tokens}")

## 3. Basic Chat Interaction

In [ ]:
# Simple chat interaction
def simple_chat(question: str) -> str:
    """Send a simple question to Claude and get a response."""
    try:
        messages = [HumanMessage(content=question)]
        response = chat_model.invoke(messages)
        return response.content
    except Exception as e:
        return f"Error: {str(e)}"

# Test basic chat
question = "What is LangChain and why is it useful?"
print(f"Question: {question}\n")
response = simple_chat(question)
print(f"Response:\n{response}")

## 4. Advanced Prompt Engineering

In [ ]:
# Create advanced prompt templates

# Template 1: Question Answering with Context
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert AI assistant specializing in {domain}. 
    Your responses should be:
    - Accurate and well-researched
    - Clear and concise
    - Include examples when appropriate
    - Cite sources or reasoning when possible"""),
    ("human", "{question}")
])

# Template 2: Creative Writing Assistant
creative_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a creative writing assistant with expertise in {genre}.
    Help users craft compelling narratives with:
    - Rich descriptions and vivid imagery
    - Strong character development
    - Engaging plot structures
    - Appropriate tone and style"""),
    ("human", "{request}")
])

# Template 3: Code Generation Assistant
code_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert {language} programmer.
    When generating code:
    - Follow best practices and design patterns
    - Include clear comments and documentation
    - Consider edge cases and error handling
    - Write clean, maintainable, and efficient code"""),
    ("human", "{task}")
])

print("✓ Prompt templates created successfully")

In [ ]:
# Demonstrate Q&A with domain expertise
qa_chain = qa_prompt | chat_model | StrOutputParser()

qa_response = qa_chain.invoke({
    "domain": "machine learning and artificial intelligence",
    "question": "Explain the difference between supervised and unsupervised learning with real-world examples."
})

print("=== Q&A Example ===\n")
print(qa_response)

In [ ]:
# Demonstrate creative writing
creative_chain = creative_prompt | chat_model | StrOutputParser()

creative_response = creative_chain.invoke({
    "genre": "science fiction",
    "request": "Write the opening paragraph of a story about an AI that discovers emotions for the first time."
})

print("=== Creative Writing Example ===\n")
print(creative_response)

In [ ]:
# Demonstrate code generation
code_chain = code_prompt | chat_model | StrOutputParser()

code_response = code_chain.invoke({
    "language": "Python",
    "task": "Create a function that implements a binary search algorithm with proper error handling."
})

print("=== Code Generation Example ===\n")
print(code_response)

## 5. Conversational Chain with Memory

In [ ]:
# Set up conversation memory storage
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Retrieve or create chat history for a session."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Create conversational prompt with memory
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful AI assistant engaged in a natural conversation.
    Remember the context of our conversation and refer back to previous messages when relevant.
    Be friendly, informative, and maintain consistency throughout the dialogue."""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create the conversational chain
conversational_chain = conversational_prompt | chat_model

# Wrap with message history
chat_with_memory = RunnableWithMessageHistory(
    conversational_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

print("✓ Conversational chain with memory initialized")

In [ ]:
# Demonstrate multi-turn conversation
def chat_with_context(message: str, session_id: str = "default") -> str:
    """Send a message and get a response with conversation history."""
    try:
        response = chat_with_memory.invoke(
            {"input": message},
            config={"configurable": {"session_id": session_id}}
        )
        return response.content
    except Exception as e:
        return f"Error: {str(e)}"

# Multi-turn conversation example
session_id = "demo_session_1"

print("=== Multi-Turn Conversation Demo ===\n")

# Turn 1
msg1 = "Hi! My name is Alex and I'm learning Python programming."
print(f"User: {msg1}")
resp1 = chat_with_context(msg1, session_id)
print(f"\nAssistant: {resp1}\n")
print("-" * 80 + "\n")

# Turn 2
msg2 = "Can you recommend some good resources for beginners?"
print(f"User: {msg2}")
resp2 = chat_with_context(msg2, session_id)
print(f"\nAssistant: {resp2}\n")
print("-" * 80 + "\n")

# Turn 3
msg3 = "What was my name again?"
print(f"User: {msg3}")
resp3 = chat_with_context(msg3, session_id)
print(f"\nAssistant: {resp3}\n")

## 6. Streaming Responses

In [ ]:
# Demonstrate streaming for real-time responses
def stream_response(message: str) -> None:
    """Stream a response token by token."""
    print("Assistant: ", end="", flush=True)
    try:
        for chunk in chat_model.stream([HumanMessage(content=message)]):
            print(chunk.content, end="", flush=True)
        print("\n")
    except Exception as e:
        print(f"\nError during streaming: {str(e)}")

print("=== Streaming Response Demo ===\n")
print("User: Tell me a short story about a robot learning to paint.\n")
stream_response("Tell me a short story about a robot learning to paint.")

## 7. Advanced Use Case: Context-Aware Research Assistant

In [ ]:
# Create a research assistant with context awareness
class ResearchAssistant:
    """An AI research assistant that maintains context across multiple queries."""
    
    def __init__(self, session_id: str = None):
        self.session_id = session_id or "research_session"
        self.chat_model = create_chat_model(temperature=0.3)  # Lower temp for factual responses
        
        # Create specialized prompt
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert research assistant helping with in-depth analysis.
            Your role is to:
            1. Provide comprehensive, well-researched answers
            2. Break down complex topics into understandable parts
            3. Remember previous queries and build upon them
            4. Ask clarifying questions when needed
            5. Suggest related topics for deeper exploration"""),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{query}")
        ])
        
        # Set up chain with memory
        chain = self.prompt | self.chat_model
        self.chain_with_memory = RunnableWithMessageHistory(
            chain,
            get_session_history,
            input_messages_key="query",
            history_messages_key="history"
        )
    
    def ask(self, query: str) -> str:
        """Ask a research question."""
        try:
            response = self.chain_with_memory.invoke(
                {"query": query},
                config={"configurable": {"session_id": self.session_id}}
            )
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"
    
    def get_history(self) -> List[Any]:
        """Get conversation history."""
        if self.session_id in store:
            return store[self.session_id].messages
        return []
    
    def clear_history(self):
        """Clear conversation history."""
        if self.session_id in store:
            store[self.session_id].clear()

print("✓ Research Assistant class created")

In [ ]:
# Demonstrate the Research Assistant
assistant = ResearchAssistant("quantum_computing_research")

print("=== Research Assistant Demo ===\n")

# Query 1: Initial question
q1 = "What is quantum computing and how does it differ from classical computing?"
print(f"Query 1: {q1}\n")
r1 = assistant.ask(q1)
print(f"Response:\n{r1}\n")
print("=" * 80 + "\n")

# Query 2: Follow-up building on context
q2 = "Can you explain more about quantum entanglement mentioned earlier?"
print(f"Query 2: {q2}\n")
r2 = assistant.ask(q2)
print(f"Response:\n{r2}\n")
print("=" * 80 + "\n")

# Query 3: Application-focused
q3 = "What are some practical applications of this technology?"
print(f"Query 3: {q3}\n")
r3 = assistant.ask(q3)
print(f"Response:\n{r3}\n")

## 8. Error Handling and Robustness

In [ ]:
# Demonstrate comprehensive error handling
class RobustChatInterface:
    """A robust chat interface with comprehensive error handling."""
    
    def __init__(self, max_retries: int = 3):
        self.max_retries = max_retries
        self.chat_model = create_chat_model()
    
    def send_message(self, message: str, retry_count: int = 0) -> Dict[str, Any]:
        """Send a message with retry logic and error handling."""
        result = {
            "success": False,
            "response": None,
            "error": None,
            "retry_count": retry_count
        }
        
        try:
            # Input validation
            if not message or not message.strip():
                raise ValueError("Message cannot be empty")
            
            if len(message) > 100000:
                raise ValueError("Message too long (max 100,000 characters)")
            
            # Send message
            response = self.chat_model.invoke([HumanMessage(content=message)])
            result["success"] = True
            result["response"] = response.content
            
        except ValueError as e:
            result["error"] = f"Validation error: {str(e)}"
        
        except Exception as e:
            error_msg = str(e)
            
            # Check if we should retry
            if retry_count < self.max_retries and "rate limit" in error_msg.lower():
                print(f"Rate limit hit, retrying... (attempt {retry_count + 1}/{self.max_retries})")
                import time
                time.sleep(2 ** retry_count)  # Exponential backoff
                return self.send_message(message, retry_count + 1)
            
            result["error"] = f"API error: {error_msg}"
        
        return result

# Test error handling
robust_chat = RobustChatInterface()

print("=== Error Handling Demo ===\n")

# Test 1: Valid message
test1 = robust_chat.send_message("What is the capital of France?")
print(f"Test 1 (Valid): Success={test1['success']}")
if test1['success']:
    print(f"Response: {test1['response'][:100]}...\n")

# Test 2: Empty message
test2 = robust_chat.send_message("")
print(f"Test 2 (Empty): Success={test2['success']}")
if not test2['success']:
    print(f"Error: {test2['error']}\n")

## 9. Practical Use Cases Gallery

In [ ]:
# Use Case 1: Technical Documentation Generator
def generate_documentation(code_snippet: str, language: str) -> str:
    """Generate comprehensive documentation for code."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a technical documentation expert.
        Generate clear, comprehensive documentation including:
        - Function/class purpose and overview
        - Parameter descriptions with types
        - Return value description
        - Usage examples
        - Edge cases and error handling notes"""),
        ("human", "Generate documentation for this {language} code:\n\n{code}")
    ])
    
    chain = prompt | chat_model | StrOutputParser()
    return chain.invoke({"language": language, "code": code_snippet})

# Example code
sample_code = """
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)
"""

print("=== Technical Documentation Generator ===\n")
docs = generate_documentation(sample_code, "Python")
print(docs)

In [ ]:
# Use Case 2: Code Review Assistant
def review_code(code: str, language: str, focus_areas: List[str] = None) -> str:
    """Perform comprehensive code review."""
    focus = ", ".join(focus_areas) if focus_areas else "all aspects"
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are an expert code reviewer.
        Review the code focusing on: {focus}
        Provide:
        - Overall assessment
        - Specific issues with line references
        - Security concerns
        - Performance considerations
        - Suggestions for improvement
        - Best practice recommendations"""),
        ("human", "Review this {{language}} code:\n\n{{code}}")
    ])
    
    chain = prompt | chat_model | StrOutputParser()
    return chain.invoke({"language": language, "code": code})

review_code_sample = """
def process_user_data(data):
    result = []
    for item in data:
        if item['age'] > 18:
            result.append(item)
    return result
"""

print("\n=== Code Review Assistant ===\n")
review = review_code(review_code_sample, "Python", ["error handling", "code quality", "performance"])
print(review)

In [ ]:
# Use Case 3: Learning Assistant with Socratic Method
class SocraticTutor:
    """A tutor that uses the Socratic method to guide learning."""
    
    def __init__(self, subject: str, session_id: str = None):
        self.subject = subject
        self.session_id = session_id or f"tutor_{subject}"
        
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", f"""You are a Socratic tutor teaching {subject}.
            Instead of giving direct answers:
            - Ask guiding questions that lead students to discover answers
            - Encourage critical thinking
            - Provide hints when students are stuck
            - Celebrate insights and correct understanding
            - Build on previous exchanges in the conversation"""),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{student_input}")
        ])
        
        chain = self.prompt | chat_model
        self.tutor_chain = RunnableWithMessageHistory(
            chain,
            get_session_history,
            input_messages_key="student_input",
            history_messages_key="history"
        )
    
    def teach(self, student_input: str) -> str:
        """Engage in Socratic dialogue with the student."""
        response = self.tutor_chain.invoke(
            {"student_input": student_input},
            config={"configurable": {"session_id": self.session_id}}
        )
        return response.content

# Demonstrate Socratic tutoring
tutor = SocraticTutor("Python programming")

print("\n=== Socratic Learning Assistant ===\n")

student1 = "I want to learn about list comprehensions in Python."
print(f"Student: {student1}\n")
teacher1 = tutor.teach(student1)
print(f"Tutor: {teacher1}\n")
print("-" * 80 + "\n")

student2 = "It's a way to create lists more efficiently?"
print(f"Student: {student2}\n")
teacher2 = tutor.teach(student2)
print(f"Tutor: {teacher2}\n")

## 10. Conversation Analysis and Insights

In [ ]:
# Analyze conversation history
def analyze_conversation(session_id: str) -> Dict[str, Any]:
    """Analyze conversation history and provide insights."""
    if session_id not in store:
        return {"error": "Session not found"}
    
    messages = store[session_id].messages
    
    analysis = {
        "total_messages": len(messages),
        "user_messages": sum(1 for m in messages if isinstance(m, HumanMessage)),
        "assistant_messages": sum(1 for m in messages if isinstance(m, AIMessage)),
        "average_user_length": 0,
        "average_assistant_length": 0,
        "topics": []
    }
    
    user_lengths = [len(m.content) for m in messages if isinstance(m, HumanMessage)]
    assistant_lengths = [len(m.content) for m in messages if isinstance(m, AIMessage)]
    
    if user_lengths:
        analysis["average_user_length"] = sum(user_lengths) / len(user_lengths)
    if assistant_lengths:
        analysis["average_assistant_length"] = sum(assistant_lengths) / len(assistant_lengths)
    
    return analysis

# Analyze a session
print("=== Conversation Analysis ===\n")
if "demo_session_1" in store:
    stats = analyze_conversation("demo_session_1")
    print(f"Session: demo_session_1")
    print(f"Total messages: {stats['total_messages']}")
    print(f"User messages: {stats['user_messages']}")
    print(f"Assistant messages: {stats['assistant_messages']}")
    print(f"Avg user message length: {stats['average_user_length']:.0f} chars")
    print(f"Avg assistant message length: {stats['average_assistant_length']:.0f} chars")
else:
    print("No conversation history found. Run the multi-turn conversation demo first.")

## 11. Interactive Chat Interface

In [ ]:
# Create an interactive chat interface
def interactive_chat_session():
    """Run an interactive chat session (for notebook use)."""
    print("=== Interactive Chat Session ===")
    print("Type 'quit' or 'exit' to end the session")
    print("Type 'clear' to clear conversation history")
    print("Type 'history' to view conversation stats\n")
    
    session_id = "interactive_session"
    
    # Note: In a Jupyter notebook, you would use input() for interactive use
    # For demo purposes, we'll show the structure
    
    example_inputs = [
        "Hello! What can you help me with?",
        "Tell me about Python decorators",
        "Can you give me an example?",
        "history",
        "quit"
    ]
    
    for user_input in example_inputs:
        print(f"\nYou: {user_input}")
        
        if user_input.lower() in ['quit', 'exit']:
            print("\nGoodbye!")
            break
        
        elif user_input.lower() == 'clear':
            if session_id in store:
                store[session_id].clear()
            print("Conversation history cleared.")
            continue
        
        elif user_input.lower() == 'history':
            stats = analyze_conversation(session_id)
            if "error" not in stats:
                print(f"\nConversation Stats:")
                print(f"  Total messages: {stats['total_messages']}")
                print(f"  Exchanges: {stats['user_messages']}")
            else:
                print("No conversation history yet.")
            continue
        
        # Get response
        response = chat_with_context(user_input, session_id)
        print(f"\nAssistant: {response}")
        print("-" * 80)

# Run demo
interactive_chat_session()

## 12. Summary and Best Practices

In [ ]:
# Summary of what we've covered
summary = """
=== LangChain Chat Application Summary ===

✓ Core Concepts Covered:
  1. Environment setup and API key management
  2. Claude model initialization and configuration
  3. Basic and advanced prompt engineering
  4. Conversational chains with memory
  5. Streaming responses
  6. Error handling and retry logic
  7. Multi-turn conversations with context
  8. Specialized assistants (Research, Socratic Tutor)
  9. Practical use cases (Documentation, Code Review)
  10. Conversation analysis and insights

✓ Best Practices:
  • Use environment variables for API keys (never hardcode)
  • Implement proper error handling and retries
  • Adjust temperature based on use case (lower for factual, higher for creative)
  • Leverage conversation memory for context-aware interactions
  • Use system prompts to define assistant behavior and constraints
  • Stream responses for better user experience with long outputs
  • Validate inputs before sending to the API
  • Monitor and analyze conversations for insights
  • Use specialized prompt templates for different tasks
  • Implement session management for multiple concurrent conversations

✓ Key LangChain Components Used:
  • ChatAnthropic: Claude model interface
  • ChatPromptTemplate: Structured prompt creation
  • MessagesPlaceholder: Dynamic message injection
  • RunnableWithMessageHistory: Conversation memory
  • InMemoryChatMessageHistory: Session storage
  • StrOutputParser: Response parsing
  • Chain composition with | operator

✓ Next Steps:
  • Explore RAG (Retrieval-Augmented Generation) for knowledge bases
  • Implement persistent storage (database) for conversation history
  • Add vector embeddings for semantic search
  • Create custom tools and function calling
  • Build multi-agent systems
  • Deploy as a web application or API
"""

print(summary)

## Appendix: Utility Functions

In [ ]:
# Utility functions for common tasks

def export_conversation(session_id: str, filename: str) -> bool:
    """Export conversation history to a text file."""
    try:
        if session_id not in store:
            print(f"Session '{session_id}' not found.")
            return False
        
        messages = store[session_id].messages
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(f"Conversation Export - Session: {session_id}\n")
            f.write(f"Generated: {__import__('datetime').datetime.now()}\n")
            f.write("=" * 80 + "\n\n")
            
            for i, msg in enumerate(messages, 1):
                role = "User" if isinstance(msg, HumanMessage) else "Assistant"
                f.write(f"[{i}] {role}:\n{msg.content}\n\n")
                f.write("-" * 80 + "\n\n")
        
        print(f"✓ Conversation exported to {filename}")
        return True
    
    except Exception as e:
        print(f"Error exporting conversation: {str(e)}")
        return False

def list_active_sessions() -> List[str]:
    """List all active chat sessions."""
    return list(store.keys())

def clear_all_sessions():
    """Clear all conversation histories."""
    store.clear()
    print("✓ All conversation histories cleared.")

print("✓ Utility functions loaded")
print("\nAvailable utilities:")
print("  • export_conversation(session_id, filename)")
print("  • list_active_sessions()")
print("  • clear_all_sessions()")